In [43]:
import requests
from bs4 import BeautifulSoup, SoupStrainer 
import sys
import string
import os
import nltk
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize 
from sklearn.feature_extraction.text import CountVectorizer
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from sklearn.cross_validation import train_test_split, KFold

###Run scraped articles through Naive Bayes classifier

In [44]:
#Get articles from saved files in directory
#Mayo list
Mayo_list = []
for article in os.listdir('/users/markregalla/desktop/metis/Project4/MayoArticles'):
    with open("/users/markregalla/desktop/metis/Project4/MayoArticles/" 
              + article, "r") as myfile:
        stuff = myfile.read()
    myfile.close
    if stuff != '':
        Mayo_list.append(stuff)
print len(Mayo_list)
#Mayo_list

35


In [45]:
#Homeopathic list
Homeo_list = []
k = 0
for article in os.listdir('/users/markregalla/desktop/metis/Project4/HomeoArticles'):
    with open('/users/markregalla/desktop/metis/Project4/HomeoArticles/' 
              + article) as myfile:
        stuff = myfile.read()
    myfile.close
    if stuff != '':
        Homeo_list.append(str(stuff.decode("ascii", "ignore")))
    k += 1
    if k == 35:
        break
#print Homeo_list
'''
total_list = []
for word in Homeo_list[0]:
    total_list.append(word.decode("ascii", "ignore"))
print total_list[0]
'''

'\ntotal_list = []\nfor word in Homeo_list[0]:\n    total_list.append(word.decode("ascii", "ignore"))\nprint total_list[0]\n'

In [112]:
new_Homeo_list = []
for i in range(len(Homeo_list)):
    new_Homeo_list.append(str(Homeo_list[i].split('\n')[1:]))
new_Homeo_list[0]

'[\'[Application Of Homoeopathic Principles In The Treatment Of Mental States]\', \' Middletown, N.Y.\', \'      \', \' THE object of this paper is to reveal\', \'      and evaluate homoeopathic principles in the treatment of mental reactions.\', \'      Such an approach is being practiced at the Middletown State homoeopathic\', \'      Hospital where our observations were made. In this paper we will also\', \'      bring forth the observations and conclusions arrived at by others as to\', \'      the effects of certain homoeopathic remedies in the treatment of certain\', \'      mental states. As Dr. Talcott once wrote, "The means used for the\', \'      recovery of patients are numerous and varied. We cannot pin our faith to a\', \'      single measure. We week to adopt all which experience has proved likely to\', \'      be beneficial." It will be out chief aim to show just what\', \'      homoeopathic remedies are most effective in mental disease, the principles\', \'      behind t

In [113]:
#new_Homeo_list[5][2]
#stuff.split('\n')[1:].decode("ascii", "ignore")
type(new_Homeo_list[0])

str

In [114]:
#Tokenize input articles by word and sentence
Mayo_word_token = []
Mayo_sent_token = []
Homeo_word_token = []
Homeo_sent_token = []

for word in Mayo_list:
    Mayo_word_token.append(word_tokenize(word))
    Mayo_sent_token.append(sent_tokenize(word))
for word in new_Homeo_list:
    Homeo_word_token.append(word_tokenize(word))
    Homeo_sent_token.append(sent_tokenize(word))

In [115]:
#Clean files and create labels
all_articles = []
all_labels = []
for i in range(len(Homeo_word_token)):
    Homeo_word_token[i]=str(Homeo_word_token[i]).replace("[","")
    Homeo_word_token[i]=str(Homeo_word_token[i]).replace("]","")
    all_articles.append(Homeo_word_token[i])
    all_labels.append('Homeo')
    Mayo_word_token[i]=str(Mayo_word_token[i]).replace("[","")
    Mayo_word_token[i]=str(Mayo_word_token[i]).replace("]","")
    all_articles.append(Mayo_word_token[i])
    all_labels.append('Mayo')

data = pd.DataFrame(columns=['Name','Text'])
data.Name=all_labels
data.Text=all_articles

In [116]:
vectorizer = CountVectorizer()  #ngram_range=(1,2))
vectorizer.fit(data.Text)
x = vectorizer.transform(data.Text)
data.head(10)

,Name,Text
0,Homeo,"'', ""'"", '', 'Application', 'Of', 'Homoeopathi..."
1,Mayo,"'Lactobacillus', 'acidophilus', 'is', 'a', 'me..."
2,Homeo,"'', ""'Aspects"", 'Of', 'Diabetic', 'Case', '', ..."
3,Mayo,"'Clear', 'gel', 'from', 'the', 'aloe', 'leaves..."
4,Homeo,"'', ""'"", 'JOSEPH', 'H.FOBES', ',', 'M.D.', ','..."
5,Mayo,"'Creatine', 'is', 'normally', 'found', 'in', '..."
6,Homeo,"'', ""'HOMOEOPATHY"", '', 'Observations', 'On', ..."
7,Mayo,"'Dehydroepiandrosterone', '(', 'DHEA', ')', 'i..."
8,Homeo,"'', ""''"", ',', ""'"", 'THANKS', 'to', 'anesthesi..."
9,Mayo,"'Dong', 'quai', 'is', 'also', 'known', 'as', '..."


In [9]:
print x

  (0, 4)	8
  (0, 5)	1
  (0, 6)	1
  (0, 13)	5
  (0, 20)	1
  (0, 23)	5
  (0, 27)	1
  (0, 30)	1
  (0, 34)	2
  (0, 38)	1
  (0, 39)	6
  (0, 46)	1
  (0, 73)	1
  (0, 75)	1
  (0, 79)	1
  (0, 80)	4
  (0, 81)	10
  (0, 82)	5
  (0, 83)	2
  (0, 107)	7
  (0, 110)	2
  (0, 113)	2
  (0, 123)	7
  (0, 144)	1
  (0, 147)	1
  :	:
  (69, 7558)	1
  (69, 7664)	1
  (69, 7684)	1
  (69, 7825)	1
  (69, 7853)	1
  (69, 8347)	1
  (69, 8653)	1
  (69, 8658)	1
  (69, 8814)	2
  (69, 9113)	1
  (69, 9248)	1
  (69, 9259)	1
  (69, 9512)	4
  (69, 9532)	1
  (69, 9533)	1
  (69, 9652)	3
  (69, 9722)	1
  (69, 9772)	2
  (69, 9880)	1
  (69, 10072)	1
  (69, 10076)	2
  (69, 10077)	1
  (69, 10081)	1
  (69, 10092)	3
  (69, 10388)	1


In [219]:
y = data.Name
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)
#print x_train, x_test, y_train, y_test

In [220]:
print type(y_test)
y_test = y_test.reset_index()
print type(y_test)
#y_test

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


In [141]:
model = MultinomialNB().fit(x_train, y_train)
results = model.predict(x_test)
print type(results)

successes = 0
for i in range(len(results)):
    print results[i], y_test.Name[i]
    if results[i] == y_test.Name[i]:
        successes += 1
print successes, i + 1
print 'Accuracy: ' + str(float(successes) / (i + 1))

<type 'numpy.ndarray'>
Homeo Homeo
Homeo Mayo
Homeo Homeo
Homeo Homeo
Homeo Homeo
Mayo Mayo
Mayo Mayo
Mayo Mayo
Homeo Homeo
Mayo Mayo
Homeo Homeo
Homeo Homeo
Mayo Mayo
Homeo Homeo
Mayo Mayo
Mayo Mayo
Homeo Homeo
Homeo Homeo
17 18
Accuracy: 0.944444444444


In [216]:
#Cross Validation

kf = KFold(35, n_folds=7)
i = 1
values = []
for train_index, test_index in kf:
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    cross_model = MultinomialNB().fit(x_train, y_train)
    results = model.predict(x_test)
    print type(results)
    y_test = np.array(y_test)
    print type(y_test)
    print 'Test ' + str(i) + ':'
    print results
    print y_test.transpose()
    values.append(results == y_test)
    i += 1

<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
Test 1:
['Homeo' 'Mayo' 'Homeo' 'Mayo' 'Homeo']
['Homeo' 'Mayo' 'Homeo' 'Mayo' 'Homeo']
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
Test 2:
['Mayo' 'Homeo' 'Mayo' 'Homeo' 'Mayo']
['Mayo' 'Homeo' 'Mayo' 'Homeo' 'Mayo']
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
Test 3:
['Homeo' 'Mayo' 'Homeo' 'Mayo' 'Homeo']
['Homeo' 'Mayo' 'Homeo' 'Mayo' 'Homeo']
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
Test 4:
['Mayo' 'Homeo' 'Mayo' 'Homeo' 'Mayo']
['Mayo' 'Homeo' 'Mayo' 'Homeo' 'Mayo']
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
Test 5:
['Homeo' 'Mayo' 'Homeo' 'Mayo' 'Homeo']
['Homeo' 'Mayo' 'Homeo' 'Mayo' 'Homeo']
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
Test 6:
['Mayo' 'Homeo' 'Homeo' 'Homeo' 'Mayo']
['Mayo' 'Homeo' 'Mayo' 'Homeo' 'Mayo']
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
Test 7:
['Homeo' 'Mayo' 'Homeo' 'Mayo' 'Homeo']
['Homeo' 'Mayo' 'Homeo' 'Mayo' 'Homeo']


In [217]:
i = 0
total_successes = []
while i < len(values):
    successes = 0
    for k in values[i]:
        if k == True:
            successes += 1
    total_successes.append(float(successes) / len(values[i]))
    print 'Test ' + str(i + 1) + ': ' + str(total_successes[i])
    i += 1
#values[5]

Test 1: 1.0
Test 2: 1.0
Test 3: 1.0
Test 4: 1.0
Test 5: 1.0
Test 6: 0.8
Test 7: 1.0


In [218]:
sum(total_successes) / len(total_successes)

0.9714285714285714